In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from dython.nominal import associations

# Read file dataset from CSV
data = pd.read_csv("../Survei Pengguna Produk Simpanan Individu.csv")

# Data detail
print(data.info())
print(data.describe(include='all'))
print(data.shape)

In [ ]:
data.rename(columns = {
    "Timestamp":"jejak waktu",
    "Kami mohon kesediaan Anda untuk menjawab pertanyaan-pertanyaan pada section berikutnya dengan jujur dan sesuai dengan produk simpanan individu yang Anda miliki. Dengan lanjut ke section berikutnya, Anda menyetujui bahwa seluruh data (terkecuali nama dan nomor telepon undian) akan digunakan sebagai bahan penelitian.":"perizinan",
    "Produk simpanan individu apa yang Anda gunakan?":"produk",
    "Nama / Inisial":"nama",
    "Nomor Telepon - GOPAY/OVO/SHOPEEPAY":"nomor_telepon",
    "Umur":"umur",
    "Domisili":"domisili",
    "Gender":"gender",
    "Status Perkawinan":"status_perkawinan",
    "Jumlah Tanggungan":"jumlah_tanggungan",
    "Kegiatan atau pekerjaan saat ini":"profesi",
    "Apa tujuan pengunaan produk simpanan individu yang Anda pilih?":"tujuan",
    "Berapa jumlah rata-rata penghasilan Anda per bulan?":"penghasilan",
    "Seberapa besar rata-rata persentase penghasilan yang Anda tabung?":"persentasi_tabungan",
    "Menurut Anda, apakah produk simpanan individu yang Anda gunakan saat ini memiliki fungsionalitas yang baik?":"rate_fungsionalitas",
    "Menurut Anda, apakah produk simpanan individu yang Anda gunakan memiliki biaya admin yang sesuai dengan fungsi yang Anda dapatkan?":"rate_admin",
    "Menurut Anda, apakah produk simpanan individu yang anda gunakan memiliki limit tabungan yang sesuai dengan kebutuhan Anda?":"rate_limit",
    "Menurut Anda, apakah produk simpanan individu yang Anda gunakan memiliki bunga tabungan yang sesuai dengan keinginan Anda?":"rate_bunga",
    "Menurut Anda, apakah produk simpanan individu yang Anda gunakan saat ini memiliki syarat setoran awal yang memberatkan?":"rate_setoran_awal",
    "Apakah produk simpanan individu yang Anda gunakan sudah sesuai dan cocok dengan kebutuhan, kemampuan, dan keinginan Anda?":"rate_kebutuhan",
    "Apakah ada alasan lain di luar bunga, limit, dan biaya admin yang membuat Anda memutuskan untuk menggunakan produk simpanan individu tersebut?":"alasan_lainnya"
    }, 
            inplace = True)

print(data)

Handle Missing Value

In [ ]:
# Cek missing value kolom
print(data.isnull().sum())

print("\n ---Visualization---")

# DataFrame untuk menghitung missing values
missing_values = pd.DataFrame({
    'Attributes': data.columns,  # Nama kolom
    'Frequency': data.isnull().sum()  # Jumlah missing values di setiap kolom
})

# Urutkan berdasarkan jumlah missing value
missing_values = missing_values.sort_values(by='Frequency', ascending=False)

# Plot visualisasi
plt.figure(figsize=(20, 10))  # Ukuran grafik
plt.title('Missing Values', fontsize=20, fontweight='bold')  # Judul grafik

sns.barplot(
    x='Attributes',
    y='Frequency',
    data=missing_values,
    order=missing_values['Attributes']
)

# Rotasi label untuk kolom agar lebih mudah dibaca
plt.xticks(rotation=45, ha='right')

# Tampilkan grafik
plt.tight_layout()  # Atur layout agar tidak terpotong
plt.show()

In [ ]:
# Hapus data kosong by label
data.dropna(subset=['produk'], inplace=True)

# Cek missing value
print(data.isnull().sum())

Handling Duplicate Data

In [ ]:
print("Data sebelum dihapus duplikat:\n", data)

# Menghapus duplikat
data = data.drop_duplicates()

print("\nData setelah dihapus duplikat:\n", data)

Remove Tabungan Valas & Optimal

In [ ]:
data = data[data["produk"] != "Tabungan Valas"]
data = data[data["produk"] != "Tabungan Optimal"]

Remove Irrelevant Attributes

In [ ]:
correlations = associations(data)
corr = correlations['corr']

In [ ]:
# tampilin nilai corr mat by label
print(corr['produk'])

In [ ]:
irrelevant_attributes = corr['produk'][corr['produk'] < 0.05].keys()
print("Kolom yang akan dihapus:", irrelevant_attributes)

# Hapus kolom dengan korelasi < 0.05
for item in irrelevant_attributes:
    # Hapus kolom dengan nama yang ada di variabel `item`
    data.drop([item], axis=1, inplace=True)

# Drop kolom alasan lainnya
data.drop("alasan_lainnya", axis=1, inplace=True)

print(data.columns)
data.to_csv("../temp/data.csv")

One Hot Encoding & Label Encoding

In [ ]:
from sklearn import preprocessing

#one-hot encoding purpose
all_purpose = {'Investasi', 'Simpanan jangka panjang', 'Kegiatan sehari-hari', 'Lainnya'}
for purpose in all_purpose:
    data[purpose] = data['tujuan'].apply(lambda x:1 if purpose in x else 0)
 
#rename column purpose    
data.rename(columns = {
    "Investasi":"investasi",
    "Simpanan jangka panjang":"simpanan_jangka_panjang",
    "Kegiatan sehari-hari":"kegiatan_sehari_hari",
    "Lainnya":"tujuan_lainnya"
    }, 
            inplace = True)
data = data.drop('tujuan',axis=1)

label_encoder = preprocessing.LabelEncoder() 

data['produk'] = label_encoder.fit_transform(data['produk'])
data['produk'].unique()

data['domisili'] = label_encoder.fit_transform(data['domisili'])
data['domisili'].unique()

data['umur'] = label_encoder.fit_transform(data['umur'])
data['umur'].unique()

data['gender'] = label_encoder.fit_transform(data['gender'])
data['gender'].unique()

data['status_perkawinan'] = label_encoder.fit_transform(data['status_perkawinan'])
data['status_perkawinan'].unique()

data['profesi'] = label_encoder.fit_transform(data['profesi'])
data['profesi'].unique()

data['penghasilan'] = label_encoder.fit_transform(data['penghasilan'])
data['penghasilan'].unique()

data['persentasi_tabungan'] = label_encoder.fit_transform(data['persentasi_tabungan'])
data['persentasi_tabungan'].unique()

data['rate_fungsionalitas'] = label_encoder.fit_transform(data['rate_fungsionalitas'])
data['rate_fungsionalitas'].unique()

data['rate_admin'] = label_encoder.fit_transform(data['rate_admin'])
data['rate_admin'].unique()

data['rate_limit'] = label_encoder.fit_transform(data['rate_limit'])
data['rate_limit'].unique()

data['rate_bunga'] = label_encoder.fit_transform(data['rate_bunga'])
data['rate_bunga'].unique()

data['rate_setoran_awal'] = label_encoder.fit_transform(data['rate_setoran_awal'])
data['rate_setoran_awal'].unique()

data['rate_kebutuhan'] = label_encoder.fit_transform(data['rate_kebutuhan'])
data['rate_kebutuhan'].unique()

Sampling Method

In [ ]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from collections import Counter
from imblearn.over_sampling import RandomOverSampler
from sklearn.utils import resample

In [ ]:
# #Temp Penghitungan Data
# class_data = []

# # Menampilkan jumlah record per kelas sebelum oversampling
# print("Jumlah record per kelas sebelum resampling:")
# print(data['produk'].value_counts())

# # Membuat list untuk menampung DataFrame hasil upsampling
# resampled_dfs = []

# # Melakukan upsampling hanya pada kelas yang memiliki jumlah kurang dari 50
# for class_label in data['produk'].unique():
#     class_data = data[data['produk'] == class_label]
    
#     # Cek jika jumlah data pada kelas kurang dari 50
#     if len(class_data) < 50:
#         # resampled_dfs.append(class_data)
#         class_data_upsampled = resample(class_data,
#                                         replace=True,  # Sampling dengan penggantian
#                                         n_samples=100,  # Mengambil sampel sampai mencapai 50
#                                         random_state=42)  # Untuk hasil yang konsisten
#         resampled_dfs.append(class_data_upsampled)
#     else:
#         # resampled_dfs.append(class_data)
#         if len(class_data) > 100:
#             # resampled_dfs.append(class_data)
#             class_data_downsampled = resample(class_data,
#                                         replace=True,  # Sampling dengan penggantian
#                                         n_samples=100,  # Mengambil sampel sampai mencapai 50
#                                         random_state=42)  # Untuk hasil yang konsisten
#             resampled_dfs.append(class_data_downsampled)
#         # Jika jumlah data sudah cukup (>= 50), masukkan data aslinya
#         # else:
#         #     resampled_dfs.append(class_data)

# # Gabungkan kembali semua kelas yang sudah diupsample
# data = pd.concat(resampled_dfs)

# # Menampilkan jumlah record per kelas setelah oversampling
# print("\nJumlah record per kelas setelah resampling:")
# print(data['produk'].value_counts())

# # Menampilkan DataFrame yang sudah diupsample
# print("\nDataFrame setelah resampling:")
# print(data)


In [ ]:
from sklearn.model_selection import train_test_split

y = data['produk']
X = data.drop(['produk'],axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)


Model

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

# Inisialisasi model Random Forest dengan jumlah estimators (tree) tertentu
model = RandomForestClassifier(n_estimators=100, random_state=42)

# Train Model
model.fit(X_train, y_train)

# Prediksi dengan data test
y_pred = model.predict(X_test)

# Make predictions on the test set
y_pred = model.predict(X_test)
probabilities = model.predict_proba(X_test)

# Get the class with max probability
most_compatible_classes = np.argmax(probabilities, axis=1)

# Evaluate the model
accuracy_train = accuracy_score(y_train, model.predict(X_train)) * 100
print(f"Model Accuracy Train: " + str(accuracy_train))

precision_train = precision_score(y_train, model.predict(X_train), average='weighted') * 100
print(f"Model Precision Train: " + str(precision_train))

recall_train = recall_score(y_train, model.predict(X_train), average='weighted') * 100
print(f"Model Recall Train: " + str(recall_train))

f1_train = f1_score(y_train, model.predict(X_train), average='weighted') * 100
print(f"Model F1 Train: " + str(f1_train))


accuracy = accuracy_score(y_test, y_pred) * 100
print(f"Model Accuracy Test: " + str(accuracy))

precision = precision_score(y_test, y_pred, average='weighted') * 100
print(f"Model Precision Test: " + str(precision))

recall = recall_score(y_test, y_pred, average='weighted') * 100
print(f"Model Recall Test: " + str(recall))

f1 = f1_score(y_test, y_pred, average='weighted') * 100
print(f"Model F1 Score Test: " + str(f1))


# Show the first 5 samples with their most compatible class
for i in range(50):
    print(f"Sample {i+1}: Most Compatible Class = {most_compatible_classes[i]}, Probability = {max(probabilities[i])*100:.2f}%")


Export Model

In [ ]:
import pickle

with open("../export_model/model_randomforest.pkl", "wb") as file:
    pickle.dump(model, file)

print("Model saved as model_randomforest.pkl!")